<a href="https://colab.research.google.com/github/foofoofoo97/punctuations_capital_letters_restoration/blob/main/punctuations_and_capital_letters_restoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#IMPORT LIBRARY 
import numpy as np
import tensorflow as tf
import time , random
import os
import math
import re
import pickle
from keras.models import Model
from keras.layers import Input, LSTM, Dense,Embedding,Bidirectional,Activation, dot, concatenate,TimeDistributed
from keras.initializers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

#PREPROCESSING CORPUS

In [ ]:
# TOKENIZE TEXTS AT CHAR LEVEL
def tokenize(lang):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', lower=False, num_words=None, char_level=True)
    tokenizer.fit_on_texts(lang)
    return tokenizer

In [ ]:
# PREPROCESSING
with open('/content/drive/MyDrive/346_assignment_2/2008.txt', 'r',encoding='cp1252') as f:
    # SEPARATE SENTENCES
    lines = f.read().split('\n')

# INITIALIZATION
input_texts = []
target_texts = []

# PREPARE CORPUS
# Input texts - list which store all lower case, punctuations removed sentences
# Target texts - list which store unnormalized sentences

#for each sentences stored in list, lines
for line in lines:
 
     #Convert all upper case to lower case
     input_text = line.lower()
 
     #Remove all characters except alphabet letters and numbers
     input_text = re.sub(r"[^a-zA-Z0-9]+", " ", input_text)
     
     # added for decoding purpose
     # \t for START and \n for END
     target_text = '\t'+ line + '\n' 

     # save current input_text into list, input_texts
     input_texts.append(input_text)
     # save current target_text into list, target_texts
     target_texts.append(target_text)

In [ ]:
print(input_texts[:5])
print(target_texts[:5])

['chogm tangani kesan perubahan iklim ', 'kampala uganda 25 nov mesyuarat ketua ketua kerajaan komanwel chogm 2007 yang berakhir hari ini mendesak masyarakat antarabangsa melakukan sesuatu lebih konkrit menangani kesan ancaman perubahan iklim ', 'pada masa yang sama para pemimpin yang hadir mahukan negara maju membantu negara miskin mencapai matlamat pembangunan alaf baru mdg ', 'dua isu ini terkandung dalam dua kenyataan berasingan yang dikeluarkan pada akhir mesyuarat dwi tahunan selama tiga hari itu selain pernyataan bersama secara umum ', 'seramai 35 ketua kerajaan daripada 53 negara anggota komanwel menyertai chogm 2007 yang pada awalnya dibayangi isu penggantungan pakistan sehingga kelihatan menenggelam isu lain ']
['\tCHOGM tangani kesan perubahan iklim \n', '\tKAMPALA, Uganda 25 Nov. – Mesyuarat Ketua-Ketua Kerajaan Komanwel (CHOGM) 2007 yang berakhir hari ini, mendesak masyarakat antarabangsa melakukan sesuatu lebih konkrit menangani kesan ancaman perubahan iklim.\n', '\tPada 

In [ ]:
# TEXT TOKENIZATION USING KERAS PREPROCESSING TEXT
# enc_token = save tokenized input characters
enc_token = tokenize(input_texts)
# reverse enc_token
reverse_enc_dict = {i: c for c, i in enc_token.word_index.items()}

# dec_token = save tokenized target characters
dec_token = tokenize(target_texts)
# reverse dec_token
reverse_dec_dict = {i: c for c, i in dec_token.word_index.items()}

# FOR TRAINING AND TESTING PURPOSE
# Calculate values for number of encoder tokens, number of decoder tokens, max encoder sequence length and max decoder sequence length
num_encoder_tokens = len(enc_token.word_index)+1  # add 1 for padding
num_decoder_tokens = len(dec_token.word_index)+1  # add 1 for padding
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#print results
print('Number of samples:', len(input_texts))
print('Target Characters:')
print(enc_token.word_index)
print('Input Characters:')
print(dec_token.word_index)
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 562574
Target Characters:
{'a': 1, ' ': 2, 'n': 3, 'e': 4, 'i': 5, 'k': 6, 'u': 7, 'r': 8, 't': 9, 'm': 10, 's': 11, 'd': 12, 'g': 13, 'l': 14, 'p': 15, 'b': 16, 'h': 17, 'y': 18, 'o': 19, 'j': 20, 'c': 21, 'w': 22, 'f': 23, '0': 24, '1': 25, '2': 26, 'z': 27, 'v': 28, '3': 29, '5': 30, '9': 31, '4': 32, '8': 33, '6': 34, '7': 35, 'q': 36, 'x': 37}
Input Characters:
{'a': 1, ' ': 2, 'n': 3, 'e': 4, 'i': 5, 'u': 6, 'k': 7, 'r': 8, 't': 9, 'm': 10, 'd': 11, 's': 12, 'g': 13, 'l': 14, 'p': 15, 'b': 16, 'h': 17, 'y': 18, 'o': 19, '\t': 20, '\n': 21, 'j': 22, '.': 23, ',': 24, 'M': 25, 'A': 26, 'S': 27, 'P': 28, 'w': 29, 'c': 30, '-': 31, 'K': 32, 'B': 33, '0': 34, 'I': 35, 'T': 36, 'f': 37, 'D': 38, 'R': 39, 'N': 40, '1': 41, '2': 42, 'U': 43, 'L': 44, 'J': 45, ')': 46, '(': 47, 'H': 48, 'z': 49, "'": 50, 'C': 51, 'O': 52, '3': 53, 'E': 54, '5': 55, '"': 56, '9': 57, 'v': 58, 'F': 59, '4': 60, '8': 61, 'G': 62, '6': 63, '7': 64, 'W': 65, '“': 66, '‘': 67, 'Y': 68, '”': 6

In [ ]:
# FOR LATER TESTING PURPOSE
# SAVE IN A PICKLE FILE
a = {'max_encoder_seq_length': max_encoder_seq_length,
     'max_decoder_seq_length': max_decoder_seq_length,
     'enc_vocab_size': num_encoder_tokens,
     'dec_vocab_size': num_decoder_tokens,
     'reverse_enc_dict': reverse_enc_dict,
     'reverse_dec_dict': reverse_dec_dict,
     'enc_token': enc_token,
     'dec_token': dec_token}

with open('/content/drive/MyDrive/346_assignment_2/params.pkl', 'wb') as handle:
   pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('SAVED IN PICKLE FILE')

SAVED IN PICKLE FILE


# TRAINING

In [ ]:
#READ VALIDATION INPUT DATA
with open('/content/drive/MyDrive/346_assignment_2/norm.txt', 'r',encoding='cp1252') as f:

  normalized_texts = f.read().split('\n')

print(normalized_texts[:20])

['berkoban adalah satu bentuk ujian daripada allah s.w.t', 'kerana dalam menjalani kehidupan di dunia ini kita harus melakukan pengorbanan', 'hidup tanpa pengorbanan adalah kehidupan yang gagal', 'dalam menjalani hidup yang baik kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan', 'a merupakan tuntutan hidup kata pensyarah jabatan dakwah dan pembangunan insan akademi pengajian islam universiti malaya roslan mohamed mengenai konsep berkorban dalam islam', 'korban berasal daripada perkataan arab iaitu udhiyah', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu nabi ibrahim dan isterinya hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya nabi ismail demi kasih dan cinta kepada allah s.w.t', 'ini adalah nilai pengorbanan yang cukup tinggi di sisi allah s.w.t', 'kerana ia bukan sahaja menguji kesabaran nabi ibrahim tetapi menduga sejauh mana ketakwaan nabi ibrahim', 'maka allah s.w.t menggantika

In [ ]:
#READ VALIDATION TARGET DATA
with open('/content/drive/MyDrive/346_assignment_2/unnorm.txt', 'r',encoding='cp1252') as f:

  unnormalized_texts = f.read().split('\n')

print(unnormalized_texts[:20])

['"Berkoban adalah satu bentuk ujian daripada Allah s.w.t. ', 'kerana dalam menjalani kehidupan di dunia ini, kita harus melakukan pengorbanan. ', 'Hidup tanpa pengorbanan adalah kehidupan yang gagal.', 'Dalam menjalani hidup yang baik, kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan. ', 'a merupakan tuntutan hidup," kata pensyarah Jabatan Dakwah dan Pembangunan Insan, Akademi Pengajian Islam, Universiti Malaya, Roslan Mohamed mengenai konsep berkorban dalam Islam.', 'Korban berasal daripada perkataan Arab iaitu udhiyah.', 'a menjurus kepada peristiwa pengorbanan satu keluarga iaitu Nabi Ibrahim dan isterinya, Hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi iaitu anaknya, Nabi Ismail demi kasih dan cinta kepada Allah s.w.t.', '"Ini adalah nilai pengorbanan yang cukup tinggi di sisi Allah s.w.t. ', 'kerana ia bukan sahaja menguji kesabaran Nabi Ibrahim tetapi menduga sejauh mana ketakwaan Nabi Ibrahim. ', 'Mak

In [ ]:
#to generate one hot sequence for input text
#x-axis = unique character value, y-axis =number in sequence
def get_text_encodings_gen(texts):

    enc_seq = enc_token.texts_to_sequences(texts)
    pad_seq = pad_sequences(enc_seq, maxlen=max_encoder_seq_length,
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=num_encoder_tokens)
    return pad_seq

In [ ]:
#to generate one hot sequence for target text
def get_text_decodings_gen(texts):
    enc_seq = dec_token.texts_to_sequences(texts)
    pad_seq = pad_sequences(enc_seq, maxlen=max_decoder_seq_length,
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=num_decoder_tokens)
  
    return pad_seq

In [ ]:
# GENERATE BATCH DATA
# Generate small batch size of data to prevent RAM from crashing 
# if np.zeros capture all training data (all 562574 sentences) will cause RAM to crash
def generate_batch(X, y, batch_size=64):
    while True:
        for j in range(0, len(X), batch_size):
            
            encoder_input_data = np.zeros((batch_size, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

            decoder_input_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

            decoder_target_data = np.zeros((batch_size, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                # Each line in tensor will input one hot sequence for each encoder_input_data, decoder_input_data and decoder_target_data
                encoder_input_data[i] = get_text_encodings_gen([input_text])
                decoder_input_data[i] = get_text_decodings_gen([target_text])
                decoder_target_data[i] = get_text_decodings_gen([target_text[1:]])

            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
#To build a sequence to sequence model for char level text unnormalization

encoder_inputs = Input(shape=(None,num_encoder_tokens))
encoder = Bidirectional(LSTM(128,return_sequences=True, return_state=True),merge_mode='concat')
encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

encoder_h = concatenate([forward_h, backward_h])
encoder_c = concatenate([forward_c, backward_c])

decoder_inputs = Input(shape=(None, num_decoder_tokens,))
decoder_lstm = LSTM(256, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[encoder_h, encoder_c])

attention = dot([decoder_outputs, encoder_outputs], axes=(2, 2))
attention = Activation('softmax', name='attention')(attention)
context = dot([attention, encoder_outputs], axes=(2, 1))
decoder_combined_context = concatenate([context, decoder_outputs])

output = TimeDistributed(Dense(128, activation="relu"))(decoder_combined_context)
output = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))(output)

model = Model([encoder_inputs, decoder_inputs], [output])

In [ ]:
#To view model structure summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 38)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, None, 256),  171008      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 114)]  0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]          

In [ ]:
print('INPUT TEXTS LENGTH' , len(input_texts))

INPUT TEXTS LENGTH 562574


In [ ]:
# Run training
def train_model(batch_size = 128, epochs=10):

    #initialize Adam optimizer
    tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,)

    #compile model by setting up optimizer, loss function and metrics
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #train with input_sentences and target_sentences
    #validate with normalized_texts and unnormalized_texts
    model.fit_generator(
        generator=generate_batch(X=input_texts, y=target_texts, batch_size=batch_size),
        steps_per_epoch=math.ceil(len(input_texts)/batch_size),
        epochs=epochs,
        verbose=1,
        validation_data = generate_batch(X=normalized_texts, y=unnormalized_texts, batch_size=batch_size),
        validation_steps=math.ceil(len(normalized_texts)/batch_size),
        workers=1,
    )


In [ ]:
#START TRAINING MODEL
train_model(epochs=5)

print('MODEL TRAINED')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
4396/4396 [==============================] - 4036s 917ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.0322 - val_accuracy: 0.9566
Epoch 2/5
4396/4396 [==============================] - 4054s 922ms/step - loss: 0.0045 - accuracy: 0.9986 - val_loss: 0.0288 - val_accuracy: 0.9947
Epoch 3/5
4396/4396 [==============================] - 4054s 922ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 0.0309 - val_accuracy: 0.9950
Epoch 4/5
4396/4396 [==============================] - 4029s 916ms/step - loss: 0.0039 - accuracy: 0.9988 - val_loss: 0.0317 - val_accuracy: 0.9925
Epoch 5/5
4396/4396 [==============================] - 4024s 915ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.0319 - val_accuracy: 0.9931
MODEL TRAINED


In [ ]:
#SAVE TRAINED MODEL INTO TRAINING.H5
model.save_weights('/content/drive/MyDrive/346_assignment_2/training.h5')

print('MODEL SAVED')

MODEL SAVED


#TEXT ENCODING

In [ ]:
#to generate one hot sequence for input text
#x-axis = unique character value, y-axis =number in sequence
def get_text_encodings(texts, parameters):
    enc_seq = parameters["enc_token"].texts_to_sequences(texts)
    pad_seq = pad_sequences(enc_seq, maxlen=parameters["max_encoder_seq_length"],
                            padding='post')
    pad_seq = to_categorical(pad_seq, num_classes=parameters["enc_vocab_size"])
    return pad_seq

# GET EXTRA CHARACTERS

In [ ]:
#to obtain extra characters if lower case of characters in dec_tokens not available in enc_tokens
def get_extra_chars(parameters):
    allowed_extras = []
    for d_c, d_i in parameters["dec_token"].word_index.items():
        #print(d_c,d_i)
        if d_c.lower() not in parameters["enc_token"].word_index:
            allowed_extras.append(d_i)
    return allowed_extras

# GET MODEL INSTANCE

In [ ]:
# GET MODEL INSTANCE
def get_model_instance(parameters):
    encoder_inputs = Input(shape=(None, parameters["enc_vocab_size"],))
    encoder = Bidirectional(LSTM(128, return_sequences=True, return_state=True),
                            merge_mode='concat')
    encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)

    encoder_h = concatenate([forward_h, backward_h])
    encoder_c = concatenate([forward_c, backward_c])

    decoder_inputs = Input(shape=(None, parameters["dec_vocab_size"],))
    decoder_lstm = LSTM(256, return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[encoder_h, encoder_c])

    attention = dot([decoder_outputs, encoder_outputs], axes=(2, 2))
    attention = Activation('softmax', name='attention')(attention)
    context = dot([attention, encoder_outputs], axes=(2, 1))
    decoder_combined_context = concatenate([context, decoder_outputs])

    output = TimeDistributed(Dense(128, activation="relu"))(decoder_combined_context)
    output = TimeDistributed(Dense(parameters["dec_vocab_size"], activation="softmax"))(output)

    model = Model([encoder_inputs, decoder_inputs], [output])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

#TEXT DECODER

In [ ]:
#DECODE INPUT TEXTS
def decode(model, parameters, input_texts, allowed_extras, batch_size):
    
    #preprocess the input_texts
    for sentence in input_texts:
       #convert all case to lower case
       sentence = sentence.lower()
       #all characters except alphabet letters and numbers is replaced with space
       sentence = re.sub(r"[^a-zA-Z0-9]+", " ", sentence)
       #for 2 or more consecutive space will be replaced by a single space
       sentence=re.sub(r'[ ]{2,}',r' ',sentence) 

    input_texts_c = input_texts.copy()

    #convert input_texts to one hot sequence
    out_dict = {}
    input_sequences = get_text_encodings(input_texts, parameters)
    parameters["reverse_dec_dict"][0] = "\n"
    outputs = [""] * len(input_sequences)

    #generate one hot sequence for decoder input data
    target_text = "\t"
    target_seq = parameters["dec_token"].texts_to_sequences([target_text] * len(input_sequences))
    target_seq = pad_sequences(target_seq, maxlen=parameters["max_decoder_seq_length"],
                               padding="post")
    target_seq_hot = to_categorical(target_seq, num_classes=parameters["dec_vocab_size"])

    extra_char_count = [0] * len(input_texts)
    prev_char_index = [0] * len(input_texts)
    i = 0
    while len(input_texts) != 0:
        curr_char_index = [i - extra_char_count[j] for j in range(len(input_texts))]
        input_encodings = np.argmax(input_sequences, axis=2)

        cur_inp_list = [input_encodings[_][curr_char_index[_]] if curr_char_index[_] < len(input_texts[_]) else 0 for _
                        in range(len(input_texts))]
        
        #Use trained model to predict and adjust the sequence for words
        output_tokens = model.predict([input_sequences, target_seq_hot], batch_size=batch_size)
        sampled_possible_indices = np.argsort(output_tokens[:, i, :])[:, ::-1].tolist()
        sampled_token_indices = []
        
        #for each possible words in sentence
        for j, per_char_list in enumerate(sampled_possible_indices):
            for index in per_char_list:
                # if current word is in allowed extras
                if index in allowed_extras:
                    if parameters["reverse_dec_dict"][index] == '\n' and cur_inp_list[j] != 0:
                        continue
                    elif parameters["reverse_dec_dict"][index] != '\n' and prev_char_index[j] in allowed_extras:
                        continue
                    sampled_token_indices.append(index)
                    extra_char_count[j] += 1
                    break
                elif parameters["enc_token"].word_index[parameters["reverse_dec_dict"][index].lower()] == cur_inp_list[j]:
                      sampled_token_indices.append(index)
                      break

        sampled_chars = [parameters["reverse_dec_dict"][index] for index in sampled_token_indices]

        outputs = [outputs[j] + sampled_chars[j] for j, output in enumerate(outputs)]
        end_indices = sorted([index for index, char in enumerate(sampled_chars) if char == '\n'], reverse=True)
        for index in end_indices:
            out_dict[input_texts[index]] = outputs[index].strip()
            del outputs[index]
            del input_texts[index]
            del extra_char_count[index]
            del sampled_token_indices[index]
            input_sequences = np.delete(input_sequences, index, axis=0)
            target_seq = np.delete(target_seq, index, axis=0)
        if i == parameters["max_decoder_seq_length"] - 1 or len(input_texts) == 0:
            break
        target_seq[:, i + 1] = sampled_token_indices
        target_seq_hot = to_categorical(target_seq, num_classes=parameters["dec_vocab_size"])
        prev_char_index = sampled_token_indices
        i += 1
    outputs = [out_dict[text] for text in input_texts_c]
    return outputs

# TESTING

In [ ]:
class Testing():
    def __init__(self):
        
        # LOAD PARAMS
        with open('/content/drive/MyDrive/346_assignment_2/params.pkl', "rb") as file:
            self.parameters = pickle.load(file)
        self.parameters["reverse_enc_dict"] = {i: c for c, i in self.parameters["enc_token"].word_index.items()}

        # LOAD MODEL
        self.model = get_model_instance(self.parameters)
        self.model.load_weights('/content/drive/MyDrive/346_assignment_2/training.h5')

        #GET EXTRA CHARACTERS
        self.allowed_extras = get_extra_chars(self.parameters)

    def new_test(self, batch_size=128):
        
        #READ NORM.TXT AND SPLIT SENTENCES INTO A LIST, NORMALIZED_TEXTS
        with open('/content/drive/MyDrive/346_assignment_2/norm.txt', 'r',encoding='cp1252') as f:
             normalized_texts = f.read().split('\n')

        #DECODE NORMALIZED_TEXTS
        results =decode(self.model, self.parameters,normalized_texts[:100], self.allowed_extras, batch_size)
        
        print("Test result is also saved in test_result.txt")
        print("__________________________________________ CHAR LEVEL TEXT UNNORMALIZATION _________________________________________")
        print("__________________________________________________ GENERATED RESULT ________________________________________________\n")
        
        #WRITE TO TEXT FILE
        file = '/content/drive/MyDrive/346_assignment_2/test_result.txt'
        with open(file, 'w') as write_result:
            index =0
            #for each sentences stored in results list
            for sentence in results:
                #write sentence into test_result file + next line '\n'
                write_result.write(sentence+'\n')
                
                index=index+1

                #print current index and sentence
                print(str(index)+' '+sentence)

In [ ]:
#CALL FUNCTION
#START TESTING
Testing().new_test()

Test result is also saved in test_result.txt
__________________________________________ CHAR LEVEL TEXT UNNORMALIZATION _________________________________________
__________________________________________________ GENERATED RESULT ________________________________________________

1 Berkoban adalah satu bentuk ujian daripada Allah SWT
2 Kerana dalam menjalani kehidupan di dunia ini kita harus melakukan pengorbanan
3 Hidup tanpa pengorbanan adalah kehidupan yang gagal
4 Dalam menjalani hidup yang baik, kita mesti berkorban masa dan tenaga misalnya berusaha mencari rezeki yang halal bagi mengelakkan kemiskinan
5 A merupakan tuntutan hidup, kata Pensyarah Jabatan Dakwah dan Pembangunan Insan Akademi Pengajian Islam Universiti Malaya, Roslan Mohamed mengenai konsep berkorban dalam Islam
6 Korban berasal daripada perkataan Arab iaitu Udhiyah
7 A menjurus kepada peristiwa pengorbanan satu keluarga iaitu Nabi Ibrahim dan isterinya hajar yang sanggup mengorbankan sesuatu yang amat mereka sayangi